# Tensors

From the [programmer's guide](https://www.tensorflow.org/programmers_guide/tensors):

> TensorFlow, as the name indicates, is a framework to define and run computations involving tensors. A tensor is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes.

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
m1 = [[1.0, 2.0],
      [3.0, 4.0]]
      
m2 = np.array([[1.0, 2.0],
              [3.0, 4.0]], dtype=np.float32)
               
m3 = tf.constant([[1.0, 2.0],
     [3.0, 4.0]])

In [4]:
print(type(m1))
print(type(m2))
print(type(m3))

<type 'list'>
<type 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [5]:
t1 = tf.convert_to_tensor(m1, dtype=tf.float32)
t2 = tf.convert_to_tensor(m2, dtype=tf.float32)
t3 = tf.convert_to_tensor(m3, dtype=tf.float32)

In [6]:
print(type(t1))
print(type(t2))
print(type(t3))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


Each tensor has a data type and a shape

In [7]:
t = tf.constant([[1.0, 2.0, 3.0],
     [3.0, 4.0, 5.0]])
print(t.dtype)
print(t.get_shape())

<dtype: 'float32'>
(2, 3)


In [8]:
# In tensor flow we can think of the rank as the number of indices needed to
# specify an element.
my_image = tf.zeros([10, 299, 299, 3])  # batch x height x width x color
r = tf.rank(my_image)
with tf.Session() as sess:
    rank_val = sess.run([r])
    print(rank_val)

[4]


### Initialization

Tensor flow has some covenience methods for creating tensors initialized with zeros (tf.zero) and ones (tf.ones). Both take a shape argument.

In [9]:
a = tf.ones([5,5]) * 0.5
print(a)

Tensor("mul:0", shape=(5, 5), dtype=float32)


In [10]:
with tf.Session() as sess:
    print(sess.run([a]))

[array([[ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5]], dtype=float32)]


### Creating Operators

Here we'll some examples of creating operators. Note that we define how operations behave, which is 
distinct from computing their value.

Refer to the TensorFlow documentation for more details on the operators offered by the library. As
an example, [here](https://www.tensorflow.org/api_guides/python/math_ops) are the math operators.

In [11]:
# Negation operator
x = tf.constant([[1,2]])
negMatrix = tf.negative(x)
print(negMatrix)

Tensor("Neg:0", shape=(1, 2), dtype=int32)


### Executing Operators

From TensorFlow in Action:

> In general a session is an environment of a software system that describes how lines of code should be run.
In TensorFlow, a session sets up how the hardware (e.g. CPU, GPU) talk to each other.

The idea is you define your machine learning algorithm in a high level, declarative way, then bind it later to
its execution environment.

In [12]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(negMatrix)
    
print(result)

[[-1 -2]]


Note you can run interactive sesssions as well, where you instantiate the session which is then implicitly
in scope for all subseqyent evals.

In [13]:
sess = tf.InteractiveSession()
x = tf.constant([[1., 2.]])
negMatrix = tf.negative(x)
result = negMatrix.eval()
print(result)
sess.close()

[[-1. -2.]]


### Variables

In TensorFLow, variables are used to store values that change over time

In [14]:
sess = tf.InteractiveSession()

raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13.]
spike = tf.Variable(False)
spike.initializer.run()

for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        updater = tf.assign(spike,True)
        updater.eval()
    else:
        tf.assign(spike, False).eval()
        
    print("Spike",spike.eval())
    
sess.close()
            

('Spike', False)
('Spike', True)
('Spike', False)
('Spike', False)
('Spike', True)
('Spike', False)
('Spike', True)


### Variables Can Be Saved and Loaded

In [26]:
v = tf.Variable(tf.constant(0))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.save(sess, 'trained_variables.ckpt')

ValueError: Parent directory of trained_variables.ckpt doesn't exist, can't save.

In [25]:
import tensorflow as tf
import os

sess = tf.InteractiveSession()
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spikes = tf.Variable([False] * len(raw_data), name='spikes')
spikes.initializer.run()
saver = tf.train.Saver()
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        spikes_val = spikes.eval()
        spikes_val[i] = True
        updater = tf.assign(spikes, spikes_val)
        updater.eval()
save_path = saver.save(sess,'./spikes')
print("spikes data saved in file: %s" % save_path)
sess.close()

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: save_10/SaveV2 = SaveV2[dtypes=[DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_10/Const_0_0, save_10/SaveV2/tensor_names, save_10/SaveV2/shape_and_slices, Variable, spikes, spikes_1, spikes_2, spikes_3, spikes_4, spikes_5, spikes_6)]]

Caused by op u'save_10/SaveV2', defined at:
  File "/home/ds/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/ds/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ds/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ds/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ds/anaconda2/lib/python2.7/site-package